# A2 - Bias in Data

The goal of this assignment is to explore the concept of bias through data on Wikipedia articles - specifically, articles on political figures from a variety of countries. For this assignment, we will combine a dataset of Wikipedia articles with a dataset of country populations, and use a machine learning service called ORES to estimate the quality of each article.

We will then perform an analysis of how the coverage of politicians on Wikipedia and the quality of articles about politicians varies between countries. The analysis will consist of a series of tables that show:

1. The countries with the greatest and least coverage of politicians on Wikipedia compared to their population.
2. The countries with the highest and lowest proportion of high quality articles about politicians.
3. A ranking of geographic regions by articles-per-person and proportion of high quality articles.

### Step 1: Getting the Article and Population Data

The first step is getting the data. The Wikipedia [politicians by country dataset](https://figshare.com/articles/Untitled_Item/5513449) can be found on Figshare. Here it is called page_data.csv.
The population data is available is called WPDS_2020_data.csv. Here it is called WPDS_df. This dataset is drawn from the [world population data sheet](https://www.prb.org/international/indicator/population/table/) published by the Population Reference Bureau.

Our analysis will also use score estimates generated from ORES. You must `pip install ores` prior to running this notebook, or follow the [installation instructions](https://github.com/wikimedia/ores).

In [1]:
import pandas as pd
import numpy as np
from ores import api
from tqdm import tqdm

In [2]:
page_data_path = '../data/page_data.csv'
WPDS_path = '../data/WPDS_2020_data.csv'

### Step 2: Cleaning the Data

Both page_df and WPDS_df contain some rows we will need to filter out or ignore. We will clean the datasets here.

In [3]:
# Filter out any rows that begin with 'Template:'. These are not Wikipedia articles.
page_df = pd.read_csv(page_data_path)
page_df = page_df.loc[~page_df['page'].str.contains('Template:')]
page_df

,page,country,rev_id
1,Bir I of Kanem,Chad,355319463
10,Information Minister of the Palestinian Nation...,Palestinian Territory,393276188
12,Yos Por,Cambodia,393822005
23,Julius Gregr,Czech Republic,395521877
24,Edvard Gregr,Czech Republic,395526568
...,...,...,...
47192,Yahya Jammeh,Gambia,807482007
47193,Lucius Fairchild,United States,807483006
47194,Fahd of Saudi Arabia,Saudi Arabia,807483153
47195,Francis Fessenden,United States,807483270


Here we add a column to the WPDS_df for 'Region' as well as 'region_population' so we have a way to associate each country with its region. Then we separate the regions and countries into separate dfs.

In [4]:
WPDS_df = pd.read_csv(WPDS_path)
WPDS_df

,FIPS,Name,Type,TimeFrame,Data (M),Population
0,WORLD,WORLD,World,2019,7772.850,7772850000
1,AFRICA,AFRICA,Sub-Region,2019,1337.918,1337918000
2,NORTHERN AFRICA,NORTHERN AFRICA,Sub-Region,2019,244.344,244344000
3,DZ,Algeria,Country,2019,44.357,44357000
4,EG,Egypt,Country,2019,100.803,100803000
...,...,...,...,...,...,...
229,WS,Samoa,Country,2019,0.200,200000
230,SB,Solomon Islands,Country,2019,0.715,715000
231,TO,Tonga,Country,2019,0.099,99000
232,TV,Tuvalu,Country,2019,0.010,10000


In [5]:
# Adding the sub-region and region_population to WPDS_df.
region = ('NORTHERN AFRICA', 244344000)

regions = [('WORLD', 7772850000) , ('AFRICA', 1337918000), ('NORTHERN AFRICA', 244344000)]
for i in range(3, len(WPDS_df)):
    if WPDS_df.iloc[i]['Type'] == 'Sub-Region':
        region = (WPDS_df.iloc[i]['Name'], WPDS_df.iloc[i]['Population'])
    regions.append(region)
    
regions_tuples_df = pd.DataFrame(regions, columns=['Region', 'region_population'])
    
WPDS_df = pd.concat([WPDS_df, regions_tuples_df], axis=1)

WPDS_df

,FIPS,Name,Type,TimeFrame,Data (M),Population,Region,region_population
0,WORLD,WORLD,World,2019,7772.850,7772850000,WORLD,7772850000
1,AFRICA,AFRICA,Sub-Region,2019,1337.918,1337918000,AFRICA,1337918000
2,NORTHERN AFRICA,NORTHERN AFRICA,Sub-Region,2019,244.344,244344000,NORTHERN AFRICA,244344000
3,DZ,Algeria,Country,2019,44.357,44357000,NORTHERN AFRICA,244344000
4,EG,Egypt,Country,2019,100.803,100803000,NORTHERN AFRICA,244344000
...,...,...,...,...,...,...,...,...
229,WS,Samoa,Country,2019,0.200,200000,OCEANIA,43155000
230,SB,Solomon Islands,Country,2019,0.715,715000,OCEANIA,43155000
231,TO,Tonga,Country,2019,0.099,99000,OCEANIA,43155000
232,TV,Tuvalu,Country,2019,0.010,10000,OCEANIA,43155000


In [6]:
# Separate all UPPERCASE entries from lowercase ones. UPPERCASE names are regions and lowercase are countries.
regions_df = WPDS_df.loc[WPDS_df.Name.str.isupper() == True]
countries_df = WPDS_df.loc[WPDS_df.Name.str.isupper() == False]

In [8]:
regions_df.head(10)

,FIPS,Name,Type,TimeFrame,Data (M),Population,Region,region_population
0,WORLD,WORLD,World,2019,7772.850,7772850000,WORLD,7772850000
1,AFRICA,AFRICA,Sub-Region,2019,1337.918,1337918000,AFRICA,1337918000
2,NORTHERN AFRICA,NORTHERN AFRICA,Sub-Region,2019,244.344,244344000,NORTHERN AFRICA,244344000
10,WESTERN AFRICA,WESTERN AFRICA,Sub-Region,2019,401.115,401115000,WESTERN AFRICA,401115000
27,EASTERN AFRICA,EASTERN AFRICA,Sub-Region,2019,444.970,444970000,EASTERN AFRICA,444970000
48,MIDDLE AFRICA,MIDDLE AFRICA,Sub-Region,2019,179.757,179757000,MIDDLE AFRICA,179757000
58,SOUTHERN AFRICA,SOUTHERN AFRICA,Sub-Region,2019,67.732,67732000,SOUTHERN AFRICA,67732000
64,NORTHERN AMERICA,NORTHERN AMERICA,Sub-Region,2019,368.193,368193000,NORTHERN AMERICA,368193000
67,LATIN AMERICA AND THE CARIBBEAN,LATIN AMERICA AND THE CARIBBEAN,Sub-Region,2019,651.036,651036000,LATIN AMERICA AND THE CARIBBEAN,651036000
68,CENTRAL AMERICA,CENTRAL AMERICA,Sub-Region,2019,178.611,178611000,CENTRAL AMERICA,178611000


In [9]:
countries_df

,FIPS,Name,Type,TimeFrame,Data (M),Population,Region,region_population
3,DZ,Algeria,Country,2019,44.357,44357000,NORTHERN AFRICA,244344000
4,EG,Egypt,Country,2019,100.803,100803000,NORTHERN AFRICA,244344000
5,LY,Libya,Country,2019,6.891,6891000,NORTHERN AFRICA,244344000
6,MA,Morocco,Country,2019,35.952,35952000,NORTHERN AFRICA,244344000
7,SD,Sudan,Country,2019,43.849,43849000,NORTHERN AFRICA,244344000
...,...,...,...,...,...,...,...,...
229,WS,Samoa,Country,2019,0.200,200000,OCEANIA,43155000
230,SB,Solomon Islands,Country,2019,0.715,715000,OCEANIA,43155000
231,TO,Tonga,Country,2019,0.099,99000,OCEANIA,43155000
232,TV,Tuvalu,Country,2019,0.010,10000,OCEANIA,43155000


### Step 3: Getting Article Quality Predictions

Now we need to get the predicted quality scores for each article in the Wikipedia dataset. We're using a machine learning system called ORES. This was originally an acronym for "Objective Revision Evaluation Service" but was simply renamed “ORES”. ORES is a machine learning tool that can provide estimates of Wikipedia article quality. The article quality estimates are, from best to worst:

1. FA - Featured article
2. GA - Good article
3. B - B-class article
4. C - C-class article
5. Start - Start-class article
6. Stub - Stub-class article

These were learned based on articles in Wikipedia that were peer-reviewed using the [Wikipedia content assessment procedures](https://en.wikipedia.org/wiki/Wikipedia:Content_assessment).These quality classes are a sub-set of quality assessment categories developed by Wikipedia editors. ORES will assign one of these 6 categories to any rev_id we send it.

To get the score estimates, we will build a list of all the rev_ids in page_df and feed them one at a time to ORES. Each query will return a generator object which we will collect in a list called 'results'.

In [10]:
ores_session = api.Session("https://ores.wikimedia.org", "DATA 512 Class project <swheele@uw.edu>")

revids = list(page_df.rev_id)
results = []

for revid in revids:
    results.append(ores_session.score("enwiki", ["articlequality"], [revid]))

In [11]:
# Empty list to population with (rev_id, score) tuples.
scores = []

Here is an example of one of the generator objects that is stored in 'results'. **We will only be concerned with the 'prediction' field.**

In [12]:
for score in results[0]:
    print(score)

{'articlequality': {'score': {'prediction': 'Stub', 'probability': {'B': 0.005643168767502225, 'C': 0.005641424870624224, 'FA': 0.0010757577110297029, 'GA': 0.001543343686495854, 'Start': 0.010537503531047517, 'Stub': 0.9755588014333005}}}}


This cell populates the list `scores=[(rev_id, score)]` which stores a tuple for each rev_id, score pair. We will then convert the list `scores` to a dataframe and finally merge it with `page_df` using rev_id as the key so that each article has a score.

In [ ]:
for i in tqdm(range(len(results))):
    for score in results[i]:
        if 'error' in list(score['articlequality'].keys()):
            scores.append((revids[i], np.nan))
        else:
            scores.append((revids[i], score['articlequality']['score']['prediction']))

In [ ]:
# Convert scores which is a list of tuples to a dataframe
scores_df = pd.DataFrame(scores, columns=['rev_id', 'score'])
scores_df

In [13]:
# scores_df.to_csv('../data/scores_df.csv')
scores_df = pd.read_csv('../data/scores_df.csv')

Here we merge the scores_df with the page_df on rev_id. We use a left merge to retain all the rows of page_df. Then we will separate all the articles that ORES was unable to determine a score (score='NaN') from the articles with valid scores.

In [14]:
page_df = page_df.merge(scores_df, how='left', left_on='rev_id', right_on='rev_id')

In [15]:
page_df

,page,country,rev_id,Unnamed: 0,score
0,Bir I of Kanem,Chad,355319463,NaN,NaN
1,Information Minister of the Palestinian Nation...,Palestinian Territory,393276188,0.0,Stub
2,Yos Por,Cambodia,393822005,1.0,Stub
3,Julius Gregr,Czech Republic,395521877,2.0,Stub
4,Edvard Gregr,Czech Republic,395526568,3.0,Stub
...,...,...,...,...,...
46696,Yahya Jammeh,Gambia,807482007,46695.0,GA
46697,Lucius Fairchild,United States,807483006,46696.0,C
46698,Fahd of Saudi Arabia,Saudi Arabia,807483153,46697.0,GA
46699,Francis Fessenden,United States,807483270,46698.0,C


In [16]:
nan_scores_df = page_df[page_df['score'].isna()]
articles_df = page_df[~page_df['score'].isna()]

There are 277 articles for which ORES was unable to determine a score. We will export the dataframe containing those rows to a file called `nan_scores_df.csv`.

In [43]:
nan_scores_df.shape

(277, 5)

In [44]:
nan_scores_df.to_csv('../data/nan_scores_df.csv')

### Step 4: Combining the Datasets

We need to merge the Wikipedia data and population data together. Both have fields containing country names which we will use for the merge. After merging the data, we will find that some entries could not be merged. Either the population dataset does not have an entry for the equivalent Wikipedia country, or vise versa.

We will use an outer merge to retain all rows from both dataframes. Then we will remove any rows that are missing article, country, or score. We will save them to a CSV file called: `wp_wpds_countries-no_match.csv`

The remaining data will be consolidated into a single CSV file called: `wp_wpds_politicians_by_country.csv`.

The schema for that file looks like this:

| Column              |
|---------------------|
| country             |
| article_name        |
| revision_id         |
| article_quality_est |
| population          |
| region              |
| region_population   |

In [17]:
merged_df = articles_df.merge(countries_df, how='outer', left_on='country', right_on='Name')

merged_df

,page,country,rev_id,Unnamed: 0,score,FIPS,Name,Type,TimeFrame,Data (M),Population,Region,region_population
0,Information Minister of the Palestinian Nation...,Palestinian Territory,393276188.0,0.0,Stub,PS,Palestinian Territory,Country,2019.0,5.008,5008000.0,WESTERN ASIA,280927000.0
1,Finance Minister of the Palestinian National A...,Palestinian Territory,596181202.0,37.0,Start,PS,Palestinian Territory,Country,2019.0,5.008,5008000.0,WESTERN ASIA,280927000.0
2,Planning Minister of the Palestinian National ...,Palestinian Territory,633612729.0,70.0,Start,PS,Palestinian Territory,Country,2019.0,5.008,5008000.0,WESTERN ASIA,280927000.0
3,Hossam Arafat (politician),Palestinian Territory,680933208.0,285.0,Stub,PS,Palestinian Territory,Country,2019.0,5.008,5008000.0,WESTERN ASIA,280927000.0
4,Tawfik Tirawi,Palestinian Territory,701106976.0,614.0,Start,PS,Palestinian Territory,Country,2019.0,5.008,5008000.0,WESTERN ASIA,280927000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
46446,NaN,NaN,NaN,NaN,NaN,PF,French Polynesia,Country,2019.0,0.280,280000.0,OCEANIA,43155000.0
46447,NaN,NaN,NaN,NaN,NaN,GU,Guam,Country,2019.0,0.175,175000.0,OCEANIA,43155000.0
46448,NaN,NaN,NaN,NaN,NaN,NC,New Caledonia,Country,2019.0,0.295,295000.0,OCEANIA,43155000.0
46449,NaN,NaN,NaN,NaN,NaN,PW,Palau,Country,2019.0,0.018,18000.0,OCEANIA,43155000.0


In [18]:
# Every row that is missing a score, Name, or page will be consolidated into no_matches_df.
no_matches_df = merged_df.loc[(merged_df.score.isna()) | (merged_df.Name.isna()) | (merged_df.page.isna())]

# After dropping the no_matches, the remaining rows are valid to use for our analysis.
merged_df = merged_df.drop(index=no_matches_df.index)
merged_df = merged_df.drop(columns=['Name', 'Type', 'FIPS', 'TimeFrame', 'Data (M)', 'Unnamed: 0']).rename(columns={'page': 'article_name', 'rev_id': 'revision_id', 'score': 'article_quality_est', 'Population': 'population', 'Region': 'region'})

merged_df

,article_name,country,revision_id,article_quality_est,population,region,region_population
0,Information Minister of the Palestinian Nation...,Palestinian Territory,393276188.0,Stub,5008000.0,WESTERN ASIA,280927000.0
1,Finance Minister of the Palestinian National A...,Palestinian Territory,596181202.0,Start,5008000.0,WESTERN ASIA,280927000.0
2,Planning Minister of the Palestinian National ...,Palestinian Territory,633612729.0,Start,5008000.0,WESTERN ASIA,280927000.0
3,Hossam Arafat (politician),Palestinian Territory,680933208.0,Stub,5008000.0,WESTERN ASIA,280927000.0
4,Tawfik Tirawi,Palestinian Territory,701106976.0,Start,5008000.0,WESTERN ASIA,280927000.0
...,...,...,...,...,...,...,...
46413,Rita Sinon,Seychelles,800323154.0,Stub,98000.0,EASTERN AFRICA,444970000.0
46414,Sylvette Frichot,Seychelles,800323798.0,Stub,98000.0,EASTERN AFRICA,444970000.0
46415,May De Silva,Seychelles,800969960.0,Start,98000.0,EASTERN AFRICA,444970000.0
46416,Vincent Meriton,Seychelles,802051093.0,Stub,98000.0,EASTERN AFRICA,444970000.0


In [19]:
no_matches_df.to_csv('../data/wp_wpds_countries-no_match.csv')
merged_df.to_csv('../data/wp_wpds_politicians_by_country.csv')

### Step 5: Analysis

The analysis will consist of calculating the proportion (as a percentage) of articles-per-population and high-quality articles for each country AND for each geographic region. By "high quality" articles, in this case we mean the number of articles about politicians in a given country that ORES predicted would be in either the "FA" (featured article) or "GA" (good article) classes.

**Examples:**
- If a country has a population of 10,000 people, and you found 10 articles about politicians from that country, then the percentage of articles-per-population would be .1%.
- If a country has 10 articles about politicians, and 2 of them are FA or GA class articles, then the percentage of high-quality articles would be 20%.

For the country-level analysis, we will begin by using groupby on country and article_quality_est so that we have a count of the number of articles of each level for each country. We also want to retain the country's population.

In [20]:
groupby_country_df = merged_df.groupby(['country', 'article_quality_est']).agg({'revision_id': 'count', 'population': 'first'})

groupby_country_df

revision_id  population
country     article_quality_est                         
Afghanistan B                              8  38928000.0
            C                             46  38928000.0
            FA                             1  38928000.0
            GA                            12  38928000.0
            Start                         99  38928000.0
...                                      ...         ...
Zimbabwe    C                             17  14863000.0
            FA                             1  14863000.0
            GA                             1  14863000.0
            Start                         48  14863000.0
            Stub                          95  14863000.0

[888 rows x 2 columns]

We have 183 countries represented in our final dataset that have at least one Wikipedia article with an estimated score.

In [21]:
len(merged_df.country.unique())

183

Here we calculate the percentage of FA or GA articles per population and the percentage of high quality articles per total number of articles for that country. We will collect the results in results_by_country_df.

In [34]:
data_by_country = []
countries = merged_df.country.unique()

for country in countries:

    if (groupby_country_df.index.isin([(country, 'FA')]).any()) | (groupby_country_df.index.isin([(country, 'GA')]).any()):
        high_articles_sum = groupby_country_df.loc[(country, ['FA', 'GA']), :].revision_id.sum()
    else:
        high_articles_sum = 0

    articles_sum = groupby_country_df.loc[(country, slice(None)), :].revision_id.sum()
    country_population = groupby_country_df.loc[(country, slice(None)), :].population[0]
    
    articles_per_pop = ( articles_sum/country_population ) * 100
    high_quality = ( high_articles_sum/articles_sum ) * 100
    
    data_by_country.append([country, articles_per_pop, high_quality])

results_by_country_df = pd.DataFrame(data_by_country, columns=['country', 'articles_per_pop', 'high_quality'])
results_by_country_df

,country,articles_per_pop,high_quality
0,Palestinian Territory,0.003574,3.910615
1,Cambodia,0.001374,1.877934
2,Canada,0.002197,2.860548
3,Egypt,0.000232,4.273504
4,Pakistan,0.000461,1.472031
...,...,...,...
178,Barbados,0.004878,0.000000
179,Belize,0.003819,0.000000
180,Djibouti,0.003745,0.000000
181,Zambia,0.000136,0.000000


Now we perform a similar aggregation and calculation for the regions. The region-level statistics will be stored in results_by_region_df.

In [24]:
groupby_region_df = merged_df.groupby(['region', 'article_quality_est']).agg({'revision_id': 'count', 'region_population': 'first'})

groupby_region_df

revision_id  region_population
region         article_quality_est                                
CARIBBEAN      B                              6         43233000.0
               C                            103         43233000.0
               FA                             2         43233000.0
               GA                            11         43233000.0
               Start                        241         43233000.0
...                                         ...                ...
WESTERN EUROPE C                            394        195479000.0
               FA                            12        195479000.0
               GA                            44        195479000.0
               Start                       1281        195479000.0
               Stub                        2775        195479000.0

[114 rows x 2 columns]

We have 19 unique regions.

In [25]:
len(merged_df.region.unique())

19

In [35]:
data_by_region = []
regions = merged_df.region.unique()

for region in regions:

    if (groupby_region_df.index.isin([(region, 'FA')]).any()) | (groupby_region_df.index.isin([(region, 'GA')]).any()):
        high_articles_sum = groupby_region_df.loc[(region, ['FA', 'GA']), :].revision_id.sum()
    else:
        high_articles_sum = 0

    articles_sum = groupby_region_df.loc[(region, slice(None)), :].revision_id.sum()
    region_population = groupby_region_df.loc[(region, slice(None)), :].region_population[0]
    
    articles_per_pop = ( articles_sum/region_population ) * 100
    high_quality = ( high_articles_sum/articles_sum ) * 100
    
    data_by_region.append([region, articles_per_pop, high_quality])

results_by_region_df = pd.DataFrame(data_by_region, columns=['region', 'articles_per_pop', 'high_quality'])
results_by_region_df

,region,articles_per_pop,high_quality
0,WESTERN ASIA,0.000912,3.472493
1,SOUTHEAST ASIA,0.000305,3.613861
2,NORTHERN AMERICA,0.000516,5.470805
3,NORTHERN AFRICA,0.000368,2.113459
4,SOUTH ASIA,0.000222,1.626202
5,MIDDLE AFRICA,0.000369,2.409639
6,WESTERN EUROPE,0.002333,1.228070
7,EASTERN AFRICA,0.000562,1.398881
8,CENTRAL AMERICA,0.000864,1.490603
9,EASTERN EUROPE,0.001279,3.161844


### Step 6: Results
Below is a summary of the results:

#### 1. Top 10 countries by coverage: 10 highest-ranked countries in terms of number of politician articles as a proportion of country population

In [36]:
results_by_country_df.nlargest(10, 'articles_per_pop', keep='all')[['country', 'articles_per_pop']]

,country,articles_per_pop
99,Tuvalu,0.540000
150,Nauru,0.472727
41,San Marino,0.238235
65,Monaco,0.105263
98,Liechtenstein,0.071795
105,Marshall Islands,0.064912
87,Tonga,0.063636
68,Iceland,0.054620
169,Andorra,0.041463
174,Federated States of Micronesia,0.033962


#### 2. Bottom 10 countries by coverage: 10 lowest-ranked countries in terms of number of politician articles as a proportion of country population

In [37]:
results_by_country_df.nsmallest(10, 'articles_per_pop', keep='all')[['country', 'articles_per_pop']].head(10)

,country,articles_per_pop
6,India,0.000069
60,Indonesia,0.000077
22,China,0.000081
151,Uzbekistan,0.000082
106,Ethiopia,0.000088
181,Zambia,0.000136
165,"Korea, North",0.000140
126,Thailand,0.000168
125,Mozambique,0.000186
116,Bangladesh,0.000187


#### 3. Top 10 countries by relative quality: 10 highest-ranked countries in terms of the relative proportion of politician articles that are of GA and FA-quality

In [38]:
results_by_country_df.nlargest(10, 'high_quality', keep='all')[['country', 'high_quality']]

,country,high_quality
165,"Korea, North",22.222222
172,Saudi Arabia,12.820513
47,Romania,12.244898
163,Central African Republic,12.121212
151,Uzbekistan,10.714286
139,Mauritania,10.416667
156,Guatemala,8.433735
175,Dominica,8.333333
49,Syria,7.812500
46,Benin,7.692308


#### 4. Bottom 10 countries by relative quality: 10 lowest-ranked countries in terms of the relative proportion of politician articles that are of GA and FA-quality

In [39]:
results_by_country_df.nsmallest(10, 'high_quality', keep='all')[['country', 'high_quality']].head(10)

,country,high_quality
13,French Guiana,0.0
24,Angola,0.0
30,Finland,0.0
34,Tunisia,0.0
41,San Marino,0.0
54,Moldova,0.0
65,Monaco,0.0
77,Turkmenistan,0.0
78,Grenada,0.0
87,Tonga,0.0


#### 5. Geographic regions by coverage: Ranking of geographic regions (in descending order) in terms of the total count of politician articles from countries in each region as a proportion of total regional population

In [41]:
results_by_region_df.sort_values('articles_per_pop', ascending=False)[['region', 'articles_per_pop']]

,region,articles_per_pop
11,Channel Islands,2.187791
17,OCEANIA,0.007244
12,SOUTHERN EUROPE,0.002421
6,WESTERN EUROPE,0.002333
18,CARIBBEAN,0.001608
9,EASTERN EUROPE,0.001279
15,SOUTHERN AFRICA,0.000936
0,WESTERN ASIA,0.000912
8,CENTRAL AMERICA,0.000864
10,SOUTH AMERICA,0.000706


#### 6. Geographic regions by coverage: Ranking of geographic regions (in descending order) in terms of the relative proportion of politician articles from countries in each region that are of GA and FA-quality

In [42]:
results_by_region_df.sort_values('high_quality', ascending=False)[['region', 'high_quality']]

,region,high_quality
2,NORTHERN AMERICA,5.470805
1,SOUTHEAST ASIA,3.613861
0,WESTERN ASIA,3.472493
9,EASTERN EUROPE,3.161844
14,EAST ASIA,3.073190
16,CENTRAL ASIA,2.857143
11,Channel Islands,2.710603
5,MIDDLE AFRICA,2.409639
3,NORTHERN AFRICA,2.113459
17,OCEANIA,2.015355
